In [ ]:
# Import libraries and packages
import pandas as pd # Data manipulation
import numpy as np
import matplotlib.pyplot as plt # Plots
import seaborn as sns # Plots
import pyreadr as pr # Read R data files

# Import machine learning packages